# Machine Learning with Python: Classification (complete tutorial)
Reference. By Mauro Di Pietro. [Here](https://getpocket.com/redirect?url=https%3A%2F%2Ftowardsdatascience.com%2Fmachine-learning-with-python-classification-complete-tutorial-d2c99dc524ec). [Code](https://github.com/mdipietro09/DataScience_ArtificialIntelligence_Utils/blob/master/machine_learning/example_classification.ipynb)

## Content
- 1 Environment setup: import libraries and read data
- 2 Data Analysis: understand the meaning and the predictive power of the variables
- 3 Feature engineering: extract features from raw data
- 4 Preprocessing: data partitioning, handle missing values, encode categorical variables, scale
- Baseline (xgboost)
    - 5 Feature Selection: keep only the most relevant variables
    - 6 Model design: train, tune hyperparameters, validation, test
    - 7 Performance evaluation: read the metrics
    - 8 Explainability: understand how the model produces results

## 1 Environment setup

In [1]:
import warnings

# data libraries
import pandas as pd 
import numpy as np 

# plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# statistical tests libraries
import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sum

# machine learning libraries
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition

# explainer libraries
from lime import lime_tabular

warnings.filterwarnings(action = 'ignore')

%matplotlib inline

In [4]:
# custom python package 
from main.utility.ml_Classification_utilities import *

ImportError: cannot import name 'models' from 'tensorflow_core.keras' (unknown location)

In [ ]:
dtf = pd.read_csv('..//..//data//kaggle-titanic-train.csv')
dtf.head()

## 2 Data Analysis

In statistics, exploratory data analysis is the process of summarizing the main characteristics of a dataset to understand what the data can tell us beyond the formal modeling or hypothesis testing task.

I always start by getting an overview of the whole dataset, in particular I want to know how many categorical and numerical variables there are and the proportion of missing data. Recognizing a variable’s type sometimes can be tricky because categories can be expressed as numbers (the Survived column is made of 1s and 0s). To this end, I am going to write a simple function that will do that for us:

In [ ]:
dtf_overview(dtf, max_cat=20, figsize=(10, 5))
# dtf.info()

There are 885 rows and 12 columns:

- each row of the table represents a specific passenger (or observation) identified by PassengerId, so I’ll set it as index (or primary key of the table for SQL lovers).
- Survived is the phenomenon that we want to understand and predict (or target variable), so I’ll rename the column as “Y”. It contains two classes: 1 if the passenger survived and 0 otherwise, therefore this use case is a binary classification problem.
- Age and Fare are numerical variables while the others are categorical.
- Only Age and Cabin contain missing data.

In [ ]:
dtf = dtf.set_index("PassengerId")
dtf = dtf.rename(columns={"Survived": "Y"})

I believe visualization is the best tool for data analysis, but you need to know what kind of plots are more suitable for the different types of variable. Therefore, I’ll provide the code to plot the appropriate visualization for different examples.

First, let’s have a look at the univariate distributions (probability distribution of just one variable). A bar plot is appropriate to understand labels frequency for a single categorical variable. For example, let’s plot the target variable:

Group variables by info
- who: Sex, Age, Embarked (which port C=Cherbourg, Q=Queenstown, S=Southampton)
- wealth: Pclass, Ticket, Fare
- where: Cabin
- how many: SibSp (with siblings/spouse), Parch (with parent/children)

In [ ]:
features = []

# target variable
# Up to 300 passengers survived and about 550 didn’t, in other words the survival rate (or the population mean) is 38%.
freqdist_plot(dtf, "Y", figsize=(5,3))

In [ ]:
#-> Population mean: 38% of the passengers survived

corr = corr_matrix(dtf, method="pearson", negative=False, lst_filters=["Y"], figsize=(15,1))

In [ ]:
# Predictive Power Score
# reference: https://www.kaggle.com/frtgnn/predictive-power-score-vs-correlation
pps = pps_matrix(dtf, lst_filters=["Y"], figsize=(15,1))

In [ ]:
# Who? Sex, Age, Embarked

#--- Sex ----#
bivariate_plot(dtf, x="Age", y="Y", figsize=(10,5))

In [ ]:
#--- Pclass ----#
bivariate_plot(dtf, x="Age", y="Pclass", figsize=(10,5))

In [ ]:
#--- Pclass ----#
bivariate_plot(dtf, x="Age", y="Sex", figsize=(10,5))

When not convinced by the “eye intuition”, you can always resort to good old statistics and run a test. In this case of categorical (Y) vs numerical (Age), I would use a one-way ANOVA test. Basically, it tests whether the means of two or more independent samples are significantly different, so if the p-value is small enough (<0.05) the null hypothesis of samples means equality can be rejected.

Apparently the passengers age contributed to determine their survival. That makes sense as the lives of women and children were to be saved first in a life-threatening situation, typically abandoning ship, when survival resources such as lifeboats were limited (the “women and children first” code).

In [ ]:
#coeff, p = test_corr(dtf, x="Fare", y="Embarked")
#coeff, p = test_corr(dtf, x="Age", y="Y")
coeff, p = test_corr(dtf, x="Sex", y="Y")

In [ ]:
#-> Sex is Predictive: the surviving rate of females is higher.
features.append("Sex")

#--- Age ---#
nan_analysis(dtf, na_x="Age", y="Y", max_cat=20, figsize=(10,5))

In order to check the validity of this first conclusion, I will have to analyze the behavior of the Sex variable with respect to the target variable. This is a case of categorical (Y) vs categorical (Sex), so I’ll plot 2 bar plots, one with the amount of 1s and 0s among the two categories of Sex (male and female) and the other with the percentages.

In [ ]:
freqdist_plot(dtf, "Age", box_logscale=True, figsize=(10,5))

The passengers were, on average, pretty young: the distribution is skewed towards the left side (the mean is 30 y.o and the 75th percentile is 38 y.o.). Coupled with the outliers in the box plot, the first spike in the left tail says that there was a significant amount of children.

I’ll take the analysis to the next level and look into the bivariate distribution to understand if Age has predictive power to predict Y. This would be the case of categorical (Y) vs numerical (Age), therefore I shall proceed like this:

- split the population (the whole set of observations) into 2 samples: the portion of passengers with Y = 1 (Survived) and Y = 0 (Not Survived).
- Plot and compare densities of the two samples, if the distributions are different then the variable is predictive because the two groups have different patterns.
- Group the numerical variable (Age) in bins (subsamples) and plot the composition of each bin, if the proportion of 1s is similar in all of them then the variable is not predictive.
- Plot and compare the box plots of the two samples to spot different behaviors of the outliers.

In [ ]:
bivariate_plot(dtf, x="Age", y="Y", figsize=(15,5))

In [ ]:
coeff, p = test_corr(dtf, x="Age", y="Y")

These 3 plots are just different perspectives of the conclusion that Age is predictive. The survival rate is higher for younger passengers: there is a spike in the left tail of 1s distribution and the first bin (0–16 y.o.) contains the highest percentage of survived passengers.

When not convinced by the “eye intuition”, you can always resort to good old statistics and run a test. In this case of categorical (Y) vs numerical (Age), I would use a one-way ANOVA test. Basically, it tests whether the means of two or more independent samples are significantly different, so if the p-value is small enough (<0.05) the null hypothesis of samples means equality can be rejected

In [ ]:
#-> Age is Predictive: the Surviving rate is higher for younger passengers, there is a spike in the left tail of Y=1 
# distribution and the first bin of Age (0-16) contains the highest percentage of survived people.
features.append("Age")

In [ ]:
# It appears that they applied the "Save women and children first" code. Let's check:
cross_distributions(dtf, x1="Sex", x2="Age", y="Y", figsize=(10,5))

In [ ]:
#--- Embarked ---#
bivariate_plot(dtf, x="Embarked", y="Y", figsize=(10,5))

In [ ]:
oeff, p = test_corr(dtf, x="Embarked", y="Y")

In [ ]:
#-> Embarked is Predictive: People from port C tend to survive better (that can be because they stayed in a fortunate area
# of the ship or just because they're smarter). Since there aren't many observations, I tested the significance 
# of the correlation (Cramer cat vs cat), it passed.
features.append("Embarked")

More than 200 female passengers (75% of the total amount of women onboard) and about 100 male passengers (less than 20%) survived. To put it another way, among women the survival rate is 75% and among men is 20%, therefore Sex is predictive. Moreover, this confirms that they gave priority to women and children.

Just like before, we can test the correlation of these 2 variables. Since they are both categorical, I’d use a Chi-Square test: assuming that two variables are independent (null hypothesis), it tests whether the values of the contingency table for these variables are uniformly distributed. If the p-value is small enough (<0.05), the null hypothesis can be rejected and we can say that the two variables are probably dependent. It’s possible to calculate Cramer’s V that is a measure of correlation that follows from this test, which is symmetrical (like traditional Pearson’s correlation) and ranges between 0 and 1 (unlike traditional Pearson’s correlation there are no negative values).

Age and Sex are examples of predictive features, but not all of the columns in the dataset are like that. For instance, Cabin seems to be an useless variable as it doesn’t provide any useful information, there are too many missing values and categories.

This kind of analysis should be carried on for each variable in the dataset to decide what should be kept as potential feature and what can be dropped because not predictive (check out the link to the full code).

### Wealth? Pclass, Ticket, Fare

In [ ]:
#--- Pclass ---#
bivariate_plot(dtf, x="Pclass", y="Y", figsize=(10,5))

In [ ]:
#-> Pclass is Predctive: the richer the higher the probability of surviving.
features.append("Pclass")

In [ ]:
#--- Ticket ---#
#-> Ticket is Useless
freqdist_plot(dtf, "Ticket", top=10, figsize=(5,3))

In [ ]:
#--- Fare ---#
bivariate_plot(dtf, x="Fare", y="Y", figsize=(15,5))

In [ ]:
bivariate_plot(dtf, x="Fare", y="Pclass", figsize=(15,5))

In [ ]:
cross_distributions(dtf, x1="Pclass", x2="Fare", y="Y", figsize=(10,5))

In [ ]:
#-> Looks there is more information in the first class: who paid higher price survived better.
# I will keep it for now and exclude one of the two in the Features Selection section.
features.append("Fare")

### Where? Cabin

In [ ]:
#--- Cabin ---#
freqdist_plot(dtf, "Cabin", top=10, figsize=(5,3))

In [ ]:
# Useless like this, let's see if the variable can be clustered using the first letter of the cabin:
dtf["Cabin_section"] = dtf["Cabin"].apply(lambda x: str(x)[0])
freqdist_plot(dtf, "Cabin_section", top=10, figsize=(5,3))

In [ ]:
## 3 Feature engineering:

It’s time to create new features from raw data using domain knowledge. I will provide one example: I’ll try to create a useful feature by extracting information from the Cabin column. I’m assuming that the letter at the beginning of each cabin number (i.e. “B96”) indicates some kind of section, maybe there were some lucky sections near to lifeboats. I will summarize the observations in clusters by extracting the section of each cabin:

In [ ]:
cross_distributions(dtf, x1="Cabin_section", x2="Pclass", y="Y", figsize=(10,5))

This plot shows how survivors are distributed among cabin sections and classes (7 survivors are in section A, 35 in B, …). Most of the sections are assigned to the 1st and the 2nd classes, while the majority of missing sections (“n”) belongs to the 3rd class. I am going to keep this new feature instead of the column Cabin:

In [ ]:
coeff, p = test_corr(dtf, x="Cabin_section", y="Y")

In [ ]:
#-> Cabin_section is predictive: for now.
features.append("Cabin_section")

### How many? SibSp, Parch

In [ ]:
coeff, p = test_corr(dtf, x="SibSp", y="Y")

In [ ]:
#-> SibSp is predictive: for now.
features.append("SibSp")

In [ ]:
#--- Parch ---#
bivariate_plot(dtf, x="Parch", y="Y", figsize=(10,5))

In [ ]:
coeff, p = test_corr(dtf, x="Parch", y="Y")

In [ ]:

#-> Parch is predictive: for now.
features.append("Parch")

### Summary

In [ ]:
dtf = dtf[features+["Y"]]
dtf.head()

## 4 Preprocessing:

To do:
- Dataset partitioning
- Resample
- Treat missings
- Encode categorical data
- Scaling
- Preprocess Test data

Data preprocessing is the phase of preparing the raw data to make it suitable for a machine learning model. In particular:

1. each observation must be represented by a single raw, in other words you can’t have two rows describing the same passenger because they will be processed separately by the model (the dataset is already in such form, so ✅). Moreover, each column should be a feature, so you shouldn’t use PassengerId as a predictor, that’s why this kind of table is called “feature matrix”.

2. The dataset must be partitioned into at least two sets: the model shall be trained on a significant portion of your dataset (so-called “train set”) and tested on a smaller set (“test set”).

3. Missing values should be replaced with something, otherwise your model may freak out.

4. Categorical data must be encoded, that means converting labels into integers, because machine learning expects numbers not strings.

5. It’s good practice to scale the data, it helps to normalize the data within a particular range and speed up the calculations in an algorithm.

Alright, let’s begin by partitioning the dataset. When splitting data into train and test sets you must follow 1 basic rule: rows in the train set shouldn’t appear in the test set as well. That’s because the model sees the target values during training and uses it to understand the phenomenon. In other words, the model already knows the right answer for the training observations and testing it on those would be like cheating. I’ve seen a lot of people pitching their machine learning models claiming 99.99% of accuracy that did in fact ignore this rule. Luckily, the Scikit-learn package knows that:


In [ ]:
check = data_preprocessing(dtf, y="Y")

### Partitioning

In [ ]:
dtf_train, dtf_test = dtf_partitioning(dtf, y="Y", test_size=0.3, shuffle=False)

In [ ]:
dtf_train.head(3)

In [ ]:
dtf_test.head(3)

### Resample

`no need to resample.`

### Missing values

Next step: the Age column contains some missing data (19%) that need to be handled. In practice, you can replace missing data with a specific value, like 9999, that keeps trace of the missing information but changes the variable distribution. Alternatively, you can use the average of the column, like I’m going to do. I’d like to underline that from a Machine Learning perspective, it’s correct to first split into train and test and then replace NAs with the average of the training set only.

In [ ]:
dtf_train, age_mean = fill_na(dtf_train, x="Age")

In [ ]:
dtf_train, embarked_mode = fill_na(dtf_train, x="Embarked")

### Categorical Encoding

There are still some categorical data that should be encoded. The two most common encoders are the Label-Encoder (each unique label is mapped to an integer) and the One-Hot-Encoder (each label is mapped to a binary vector). The first one is suited for data with ordinality only. If applied to a column with no ordinality, like Sex, it would turn the vector [male, female, female, male, …] into [1, 2, 2, 1, …] and we would have that female > male and with an average of 1.5 which makes no sense. On the other hand, the One-Hot-Encoder would transform the previous example into two dummy variables (dichotomous quantitative variables): Male [1, 0, 0, 1, …] and Female [0, 1, 1, 0, …]. It has the advantage that the result is binary rather than ordinal and that everything sits in an orthogonal vector space, but features with high cardinality can lead to a dimensionality issue. I shall use the One-Hot-Encoding method, transforming 1 categorical column with n unique values into n-1 dummies. Let’s encode Sex as example:

In [ ]:
dtf_train = add_dummies(dtf_train, x="Sex", dropx=True)

In [ ]:
dtf_train = add_dummies(dtf_train, x="Embarked", dropx=True)

In [ ]:
dtf_train = add_dummies(dtf_train, x="Pclass", dropx=True)

In [ ]:
dtf_train = add_dummies(dtf_train, x="Cabin_section", dropx=True)

In [ ]:
dtf_train = pop_columns(dtf_train, ["Y"], where="end")
dtf_train.head()

### Scaling

Last but not least, I’m going to scale the features. There are several different ways to do that, I’ll present just the most used ones: the Standard-Scaler and the MinMax-Scaler. The first one assumes data is normally distributed and rescales it such that the distribution centres around 0 with a standard deviation of 1. However, the outliers have an influence when computing the empirical mean and standard deviation which shrink the range of the feature values, therefore this scaler can’t guarantee balanced feature scales in the presence of outliers. On the other hand, the MinMax-Scaler rescales the data set such that all feature values are in the same range (0–1). It is less affected by outliers but compresses all inliers in a narrow range. Since my data is not normally distributed, I’ll go with the MinMax-Scaler:

In [ ]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

In [ ]:
dtf_train, scaler = scaling(dtf_train, y="Y", scalerX=scaler, task="classification")

In [ ]:
dtf_train.head()

In [ ]:
dtf_overview(dtf_train)

### Preprocess Test data

In [ ]:
## Na
dtf_test = fill_na(dtf_test, x="Age", value=age_mean)

In [ ]:
## Categorical
dtf_test = add_dummies(dtf_test, x="Sex", dropx=True)
dtf_test = add_dummies(dtf_test, x="Embarked", dropx=True)
dtf_test = add_dummies(dtf_test, x="Pclass", dropx=True)
dtf_test = add_dummies(dtf_test, x="Cabin_section", dropx=True)

In [ ]:
## There are two columns less in the Test set: Cabin_section_T, Cabin_section_G 
dtf_test["Cabin_section_T"], dtf_test["Cabin_section_G"] = 0, 0

In [ ]:
dtf_test = dtf_test[dtf_train.columns]
dtf_test.head()

In [ ]:
## Scale
dtf_test, _ = scaling(dtf_test, y="Y", scalerX=scaler, fitted=True)

In [ ]:
dtf_overview(dtf_test)

## Baseline (xgboost)
Plan:
- Feature Selection: by correlation, by p-value, by importance
- Model Design
- Train / Test
- Evaluate
- Visualize model
- Explainability

### 5 Feature Selection:

Feature selection is the process of selecting a subset of relevant variables to build the machine learning model. It makes the model easier to interpret and reduces overfitting (when the model adapts too much to the training data and performs badly outside the train set).

I already did a first “manual” feature selection during data analysis by excluding irrelevant columns. Now it’s going to be a bit different because we assume that all the features in the matrix are relevant and we want to drop the unnecessary ones. When a feature is not necessary? Well, the answer is easy: when there is a better equivalent, or one that does the same job but better.

I’ll explain with an example: Pclass is highly correlated with Cabin_section because, as we’ve seen before, certain sections were located in 1st class and others in the 2nd. Let’s compute the correlation matrix to see it:

In [ ]:
#--- correlation ---#
corr = corr_matrix(dtf_train, method="pearson", negative=False, annotation=True, figsize=(15,7))

In [ ]:
pps = pps_matrix(dtf_train, annotation=True, figsize=(15,7))

One among Pclass and Cabin_section could be unnecessary and we may decide to drop it and keep the most useful one (i.e. the one with lowest p-value or the one that most reduces entropy).

I will show two different ways to perform automatic feature selection: first I will use a regularization method and compare it with the ANOVA test already mentioned before, then I will show how to get feature importance from ensemble methods.

LASSO regularization is a regression analysis method that performs both variable selection and regularization in order to enhance accuracy and interpretability.

In [ ]:
#--- p values ---#
dic_feat_sel = features_selection(dtf_train, y="Y", task="classification", top=10, figsize=(10,5))

The blue features are the ones selected by both ANOVA and LASSO, the others are selected by just one of the two methods.

Random forest is an ensemble method that consists of a number of decision trees in which every node is a condition on a single feature, designed to split the dataset into two so that similar response values end up in the same set. Features importance is computed from how much each feature decreases the entropy in a tree.

In [ ]:
#--- importance ---#
model = ensemble.RandomForestClassifier(n_estimators=100, criterion="entropy", random_state=0)

feat_imp = features_importance(X=dtf_train.drop("Y",axis=1).values, y=dtf_train["Y"].values, 
                               X_names=dtf_train.drop("Y",axis=1).columns.tolist(), 
                              model=model, task="classification", figsize=(15,5))

It’s really interesting that Age and Fare, that are the most important features this time, weren’t the top features before and that on the contrary Cabin_section E, F and D don’t appear really useful here.

Personally, I always try to use less features as possible, so here I select the following ones and proceed with the design, train, test and evaluation of the machine learning model:

In [ ]:
# -> selection
X_names = ["Age", "Fare", "Sex_male", "SibSp", "Pclass_3", "Parch", "Cabin_section_n", "Embarked_S", "Pclass_2",
           "Cabin_section_F", "Cabin_section_E", "Cabin_section_D"]

Please note that before using test data for prediction you have to preprocess it just like we did for the train data.

### 6 Model design

Finally, it’s time to build the machine learning model. First, we need to choose an algorithm that is able to learn from training data how to recognize the two classes of the target variable by minimizing some error function.

`'images/scikit-learn-algoritm-cheat-sheet.jpg'`

I suggest to always try a gradient boosting algorithm (like XGBoost). It’s a machine learning technique which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees. Basically it’s similar to a Random Forest with the difference that every tree is fitted on the error of the previous one.

There a lot of hyperparameters and there is no general rule about what is best, so you just have to find the right combination that fits your data better. You could do different tries manually or you can let the computer do this tedious job with a GridSearch (tries every possible combination but takes time) or with a RandomSearch (tries randomly a fixed number of iterations). I’ll try a RandonSearch for my hyperparameter tuning: the machine will iterate n times (1000) through training data to find the combination of parameters (specified in the code below) that maximizes a scoring function used as KPI (accuracy, the ratio of number of correct predictions to the total number of input samples):

In [ ]:
X_train = dtf_train[X_names].values
y_train = dtf_train["Y"].values

In [ ]:
model = ensemble.GradientBoostingClassifier()

In [ ]:
param_dic = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001],      #weighting factor for the corrections by new trees when added to the model
             'n_estimators':[100,250,500,750,1000,1250,1500,1750],  #number of trees added to the model
             'max_depth':[2,3,4,5,6,7],                             #maximum depth of the tree
             'min_samples_split':[2,4,6,8,10,20,40,60,100],         #sets the minimum number of samples to split
             'min_samples_leaf':[1,3,5,7,9],                        #the minimum number of samples to form a leaf
             'max_features':[2,3,4,5,6,7],                          #square root of features is usually a good starting point
             'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1]}            #the fraction of samples to be used for fitting the individual base learners. Values lower than 1 generally lead to a reduction of variance and an increase in bias.

Cool, that’s the best model, with a mean accuracy of 0.85, so probably 85% of predictions on the test set will be correct.

We can also validate this model using a k-fold cross-validation, a procedure that consists in splitting the data k times into train and validation sets and for each split the model is trained and tested. It’s used to check how well the model is able to get trained by some data and predict unseen data.

I’d like to clarify that I call validation set a set of examples used to tune the hyperparameters of a classifier, extracted from splitting training data. On the other end, a test set is a simulation of how the model would perform in production when it’s asked to predict observations never seen before.

It’s common to plot a ROC cruve for every fold, a plot that illustrates how the ability of a binary classifier changes as its discrimination threshold is varied. It is created by plotting the true positive rate (1s predicted correctly) against the false positive rate (1s predicted that are actually 0s) at various threshold settings. The AUC (area under the ROC curve) indicates the probability that the classifier will rank a randomly chosen positive observation (Y=1) higher than a randomly chosen negative one (Y=0).

Now I’ll show an example of with 10 folds (k=10):

In [ ]:
# this takes a while
model = tune_classif_model(X_train, y_train, model, param_dic, scoring="accuracy", 
                           searchtype="RandomSearch", n_iter=1000, cv=10, figsize=(10,5))

According to this validation, we should expect an AUC score around 0.84 when making predictions on the test.

For the purpose of this tutorial I’d say that the performance is fine and we can proceed with the model selected by the RandomSearch. Once that the right model is selected, it can be trained on the whole train set and then tested on the test set.

### Train / Test

In [ ]:
X_test = dtf_test[X_names].values
y_test = dtf_test["Y"].values

In [ ]:
model, predicted_prob, predicted = fit_classif_model(model, X_train, y_train, X_test, threshold=0.5)

In the code above I made two kinds of predictions: the first one is the probability that an observation is a 1, and the second is the prediction of the label (1 or 0). To get the latter you have to decide a probability threshold for which an observation can be considered as 1, I used the default threshold of 0.5.

### 7 Performance evaluation

Moment of truth, we’re about to see if all this hard work is worth. The whole point is to study how many correct predictions and error types the model makes.

I’ll evaluate the model using the following common metrics: Accuracy, AUC, Precision and Recall. I already mentioned the first two, but I reckon that the others are way more important. Precision is the fraction of 1s (or 0s) that the model predicted correctly among all predicted 1s (or 0s), so it can be seen as a sort of confidence level when predicting a 1 (or a 0). Recall is the portion of 1s (or 0s) that the model predicted correctly among all 1s (or 0s) in the test set, basically it’s the true 1 rate. Combining Precision and Recall with an armonic mean, you get the F1-score.

Let’s see how the model did on the test set:

In [ ]:
evaluate_classif_model(y_test, predicted, predicted_prob, figsize=(20,5))

As expected, the general accuracy of the model is around 85%. It predicted 71% of 1s correctly with a precision of 84% and 92% of 0s with a precision of 85%. In order to understand these metrics better, I’ll break down the results in a confusion matrix:

We can see that the model predicted 85 (70+15) 1s of which 70 are true positives and 15 are false positives, so it has a Precision of 70/85 = 0.82 when predicting 1s. On the other hand, the model got 70 1s right of all the 96 (70+26) 1s in the test set, so its Recall is 70/96 = 0.73.

Choosing a threshold of 0.5 to decide whether a prediction is a 1 or 0 led to this result. Would it be different with another one? Definitely yes, but there is no threshold that would bring the top score on both precision and recall, choosing a threshold means to make a compromise between these two metrics. I’ll show what I mean by plotting the ROC curve and the precision-recall curve of the test result:

Every point of these curves represents a confusion matrix obtained with a different threshold (the numbers printed on the curves). I could use a threshold of 0.1 and gain a recall of 0.9, meaning that the model would predict 90% of 1s correctly, but the precision would drop to 0.4, meaning that the model would predict a lot of false positives. So it really depends on the type of use case and in particular whether a false positive has an higher cost of a false negative.

When the dataset is balanced and metrics aren’t specified by project stakeholder, I usually choose the threshold that maximize the F1-score. Here’s how:

Before moving forward with the last section of this long tutorial, I’d like to say that we can’t say that the model is good or bad yet. The accuracy is 0.85, is it high? Compared to what? You need a baseline to compare your model with. Maybe the project you’re working on is about building a new model to replace an old one that can be used as baseline, or you can train different machine learning models on the same train set and compare the performance on a test set.

### Visualize model

In [ ]:
model2d = ensemble.GradientBoostingClassifier()
model2d.set_params(**{'subsample':1, 'n_estimators':1750, 'min_samples_split':6, 'min_samples_leaf':1, 'max_depth':4, 'learning_rate':0.001})

In [ ]:
plot2d_classif_model(X_train, y_train, X_test, y_test, model2d, annotate=False, figsize=(10,5))

### 8 Explainability

You analyzed and understood the data, you trained a model and tested it, you’re even satisfied with the performance. You think you’re done? Wrong. High chance that the project stakeholder doesn’t care about your metrics and doesn’t understand your algorithm, so you have to show that your machine learning model is not a black box.

The Lime package can help us to build an explainer. To give an illustration I will take a random observation from the test set and see what the model predicts:

In [ ]:
i = 4
print("True:", y_test[i], "--> Pred:", predicted[i], "| Prob:", np.max(predicted_prob[i]))

exp = explainer(X_train, X_names, model, y_train,  X_test_instance=X_test[i], top=10, task="classification")

In [ ]:

The main factors for this particular prediction are that the passenger is female (Sex_male = 0), young (Age ≤ 22) and traveling in 1st class (Pclass_3 = 0 and Pclass_2 = 0).

The confusion matrix is a great tool to show how the testing went, but I also plot the classification regions to give a visual aid of what observations the model predicted correctly and what it missed. In order to plot the data in 2 dimensions some dimensionality reduction is required (the process of reducing the number of features by obtaining a set of principal variables). I will give an example using the PCA algorithm to summarize the data into 2 variables obtained with linear combinations of the features.

In [ ]:
## PCA    
pca = decomposition.PCA(n_components=2)    
X_train_2d = pca.fit_transform(X_train)    
X_test_2d = pca.transform(X_test)## train 2d model    
model_2d = ensemble.GradientBoostingClassifier()    
model_2d.fit(X_train, y_train)

## plot classification regions    
from matplotlib.colors import ListedColormap    
colors = {np.unique(y_test)[0]:"black", np.unique(y_test)[1]:"green"}    
X1, X2 = np.meshgrid(np.arange(start=X_test[:,0].min()-1, stop=X_test[:,0].max()+1, step=0.01),    
np.arange(start=X_test[:,1].min()-1, stop=X_test[:,1].max()+1, step=0.01))    
fig, ax = plt.subplots()    
Y = model_2d.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape)    
ax.contourf(X1, X2, Y, alpha=0.5, cmap=ListedColormap(list(colors.values())))    
ax.set(xlim=[X1.min(),X1.max()], ylim=[X2.min(),X2.max()], title="Classification regions")    

for i in np.unique(y_test):    
    ax.scatter(X_test[y_test==i, 0], X_test[y_test==i, 1],     
               c=colors[i], label="true "+str(i))      
plt.legend()    
plt.show()

## Conclusion

This article has been a tutorial to demonstrate how to approach a classification use case with data science. I used the Titanic dataset as an example, going through every step from data analysis to the machine learning model.

In the exploratory section, I analyzed the case of a single categorical variable, a single numerical variable and how they interact together. I gave an example of feature engineering extracting a feature from raw data. Regarding preprocessing, I explained how to handle missing values and categorical data. I showed different ways to select the right features, how to use them to build a machine learning classifier and how to assess the performance. In the final section, I gave some suggestions on how to improve the explainability of your machine learning model.

An important note is that I haven’t covered what happens after your model is approved for deployment. Just keep in mind that you need to build a pipeline to automatically process new data that you will get periodically.

Now that you know how to approach a data science use case, you can apply this code and method to any kind of binary classification problem, carry out your own analysis, build your own model and even explain it.